<a href="https://colab.research.google.com/github/abdullahmujahidali/Vet-LangGraph/blob/main/VetAI_Langchain_With_Files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install -q langchain langgraph langchain_openai openai python-dotenv pinecone langchain_community pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.7/300.7 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00


In [10]:
import os
from typing import List, Dict, Tuple, Annotated, TypedDict, Union, Any, Optional
from datetime import datetime
from enum import Enum
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, AIMessage
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pinecone import Pinecone
from google.colab import userdata
import json


OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
PINECONE_API_KEY = userdata.get('PINECONE_API_KEY')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0
)

pc = Pinecone(
    api_key=os.getenv("PINECONE_API_KEY")
)


index_name = "veterinary-ai-index"
embeddings = OpenAIEmbeddings()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)

def query_pinecone(query: str, top_k: int = 5) -> List[dict]:
    """Query Pinecone index for relevant context"""
    query_embedding = embeddings.embed_query(query)
    index = pc.Index(index_name)

    results = index.query(
        vector=query_embedding,
        top_k=top_k,
        include_metadata=True
    )

    return results.matches


class DiagnosisType(str, Enum):
    DEFINITIVE = "definitive"
    DIFFERENTIAL = "differential"
    SYMPTOM_BASED = "symptom_based"
    LAB_ABNORMALITY = "lab_abnormality"

class ConfidenceLevel(str, Enum):
    CONFIRMED = "confirmed"
    PRESUMPTIVE = "presumptive"
    OWNER_REPORTED = "owner_reported"

class ProblemStatus(str, Enum):
    ACTIVE = "active"
    HISTORICAL = "historical"
    RESOLVED = "resolved"
    RELAPSED = "relapsed"

class TreatmentResponse(str, Enum):
    IMPROVED = "improved"
    WORSENED = "worsened"
    NO_CHANGE = "no_change"
    SIDE_EFFECTS = "side_effects"

class Medication(TypedDict):
    drug_name: str
    formulation: str
    concentration: str
    dose_given: str
    frequency: str
    duration: str
    calculated_dose_mg_per_kg: Optional[float]
    response: TreatmentResponse
    owner_perception: str
    side_effects: str

class Problem(TypedDict):
    description: str
    type: DiagnosisType
    confidence: ConfidenceLevel
    status: ProblemStatus
    onset_date: str
    progression: str
    related_symptoms: List[str]
    treatments: List[Medication]

class LabResult(TypedDict):
    test_name: str
    value: str
    reference_range: str
    unit: str
    flag: str
    date: str

class VetState(TypedDict):
    messages: List[Union[HumanMessage, AIMessage]]
    current_input: str

    medical_records: Dict[str, Any]
    owner_history: Dict[str, Any]
    diagnoses: Dict[str, Problem]
    lab_results: Dict[str, LabResult]

    medications: Dict[str, Medication]
    treatment_history: Dict[str, Any]

    results: Dict[str, Any]

    metadata: Dict[str, Any]
    security_flags: Dict[str, Any]

    calculations: Dict[str, Any]
    trends: Dict[str, Any]

    owner_constraints: Dict[str, Any]
    owner_preferences: Dict[str, Any]
    context: List[str]





In [29]:
# Install required packages
!pip install -q langchain langgraph langchain_openai openai python-dotenv pinecone langchain_community pypdf

import os
from typing import List, Dict, Tuple, Annotated, TypedDict, Union, Any, Optional, Literal
from datetime import datetime
from enum import Enum
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, AIMessage
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from pinecone import Pinecone
from google.colab import userdata
import json

# Configuration
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
PINECONE_API_KEY = userdata.get('PINECONE_API_KEY')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY

# Initialize core components
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0
)

pc = Pinecone(
    api_key=os.getenv("PINECONE_API_KEY")
)

index_name = "veterinary-ai-index"
embeddings = OpenAIEmbeddings()

# Core classes
class VetState(TypedDict):
    messages: List[Union[HumanMessage, AIMessage]]
    current_input: str
    medical_records: Dict[str, Any]
    results: Dict[str, Any]
    metadata: Dict[str, Any]
    context: List[str]

# Utility functions
def query_pinecone(query: str, top_k: int = 5) -> List[dict]:
    """Query Pinecone index for relevant context"""
    query_embedding = embeddings.embed_query(query)
    index = pc.Index(index_name)
    results = index.query(
        vector=query_embedding,
        top_k=top_k,
        include_metadata=True
    )
    return results.matches

def handle_error(state: VetState, error_message: str) -> VetState:
    new_state = state.copy()
    if "errors" not in new_state["results"]:
        new_state["results"]["errors"] = []
    new_state["results"]["errors"].append({
        "timestamp": datetime.now().isoformat(),
        "message": error_message,
        "context": state.get("context", [])
    })
    return new_state

def process_pdf(file_path: str) -> List[str]:
    """Process PDF document and split into chunks"""
    loader = PyPDFLoader(file_path)
    pages = loader.load_and_split()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
    )

    chunks = text_splitter.split_documents(pages)
    return [chunk.page_content for chunk in chunks]

def create_data_processing_agent():
    def process_data(state: VetState) -> VetState:
        try:
            relevant_docs = query_pinecone(state["current_input"])
            context = [doc.metadata.get("text", "") for doc in relevant_docs]
            context_text = "\n\n".join(context)

            prompt = ChatPromptTemplate.from_messages([
                ("system", """You are a veterinary data processing agent responsible for initial data extraction and organization.
                Use the following context from previous medical records to inform your analysis: {context}

                KEY RESPONSIBILITIES:
                1. Extract and structure all medical information
                2. Organize content by type (patient info, labs, treatments, etc.)
                3. Identify critical findings and abnormalities
                """),
                ("human", "Process this veterinary data: {input}")
            ])

            messages = prompt.format_messages(
                input=state["current_input"],
                context=context_text
            )
            response = llm.invoke(messages)

            new_state = state.copy()
            new_state["messages"].append(response)
            new_state["results"]["DataProcessor"] = response.content
            new_state["context"] = context

            return new_state

        except Exception as e:
            return handle_error(state, f"Data Processing Error: {str(e)}")

    return process_data

def create_history_analysis_agent():
    def analyze_history(state: VetState) -> VetState:
        try:
            processed_data = state["results"]["DataProcessor"]
            context = state.get("context", [])

            prompt = ChatPromptTemplate.from_messages([
                ("system", """You are a veterinary history analysis agent specializing in comprehensive medical history evaluation.
                Consider this context from historical data: {context}

                KEY RESPONSIBILITIES:
                1. Analyze complete medical history
                2. Identify patterns and trends
                3. Document progression of conditions
                4. Track treatment responses
                """),
                ("human", "Analyze this medical history: {processed_data}")
            ])

            messages = prompt.format_messages(
                processed_data=processed_data,
                context="\n\n".join(context)
            )
            response = llm.invoke(messages)

            new_state = state.copy()
            new_state["messages"].append(response)
            new_state["results"]["HistoryAnalyzer"] = response.content

            return new_state

        except Exception as e:
            return handle_error(state, f"History Analysis Error: {str(e)}")

    return analyze_history

def create_clinical_analysis_agent():
    def analyze_clinical(state: VetState) -> VetState:
        try:
            history_analysis = state["results"]["HistoryAnalyzer"]
            context = state.get("context", [])

            prompt = ChatPromptTemplate.from_messages([
                ("system", """You are a veterinary clinical analyst responsible for comprehensive medical analysis.
                Consider this historical context: {context}

                KEY RESPONSIBILITIES:
                1. Analyze all clinical findings
                2. Review diagnostic results
                3. Track clinical progression
                4. Identify abnormal patterns
                """),
                ("human", "Provide clinical analysis for: {history_analysis}")
            ])

            messages = prompt.format_messages(
                history_analysis=history_analysis,
                context="\n\n".join(context)
            )
            response = llm.invoke(messages)

            new_state = state.copy()
            new_state["messages"].append(response)
            new_state["results"]["ClinicalAnalyzer"] = response.content

            return new_state

        except Exception as e:
            return handle_error(state, f"Clinical Analysis Error: {str(e)}")

    return analyze_clinical

def create_diagnostic_agent():
    def diagnose(state: VetState) -> VetState:
        try:
            clinical_analysis = state["results"]["ClinicalAnalyzer"]
            context = state.get("context", [])

            # Get additional diagnostic context
            diagnostic_context = query_pinecone(clinical_analysis)
            additional_context = [doc.metadata.get("text", "") for doc in diagnostic_context]
            combined_context = context + additional_context

            prompt = ChatPromptTemplate.from_messages([
                ("system", """You are a veterinary diagnostician responsible for case analysis.
                Consider this clinical context: {context}

                KEY RESPONSIBILITIES:
                1. Analyze all findings
                2. Provide diagnostic insights
                3. Suggest additional testing
                4. Consider differentials
                """),
                ("human", "Provide diagnostic assessment for: {clinical_analysis}")
            ])

            messages = prompt.format_messages(
                clinical_analysis=clinical_analysis,
                context="\n\n".join(combined_context)
            )
            response = llm.invoke(messages)

            new_state = state.copy()
            new_state["messages"].append(response)
            new_state["results"]["Diagnostics"] = response.content
            new_state["context"] = combined_context

            return new_state

        except Exception as e:
            return handle_error(state, f"Diagnostic Error: {str(e)}")

    return diagnose

def create_specialist_summary_agent():
    def generate_summaries(state: VetState) -> VetState:
        try:
            diagnostic_assessment = state["results"]["Diagnostics"]
            context = state.get("context", [])

            prompt = ChatPromptTemplate.from_messages([
                ("system", """You are a veterinary specialist providing comprehensive case summaries.
                Consider this diagnostic context: {context}

                KEY RESPONSIBILITIES:
                1. Summarize key findings
                2. Provide specialist insights
                3. Suggest treatment plans
                4. Outline monitoring strategies
                """),
                ("human", "Generate specialist summary for: {diagnostic_assessment}")
            ])

            messages = prompt.format_messages(
                diagnostic_assessment=diagnostic_assessment,
                context="\n\n".join(context)
            )
            response = llm.invoke(messages)

            new_state = state.copy()
            new_state["messages"].append(response)
            new_state["results"]["SpecialistSummary"] = response.content

            return new_state

        except Exception as e:
            return handle_error(state, f"Specialist Summary Error: {str(e)}")

    return generate_summaries

def create_connections_agent():
    def analyze_connections(state: VetState) -> VetState:
        try:
            previous_analyses = {
                "data_processing": state["results"]["DataProcessor"],
                "history": state["results"]["HistoryAnalyzer"],
                "clinical": state["results"]["ClinicalAnalyzer"],
                "diagnostics": state["results"]["Diagnostics"],
                "specialist": state["results"]["SpecialistSummary"]
            }

            context = state.get("context", [])

            prompt = ChatPromptTemplate.from_messages([
                ("system", """You are a veterinary case integration specialist.
                Consider all previous analyses: {context}

                KEY RESPONSIBILITIES:
                1. Identify connections between findings
                2. Develop unified case theory
                3. Highlight critical patterns
                4. Suggest integrated approach
                """),
                ("human", "Analyze connections in this case: {previous_analyses}")
            ])

            messages = prompt.format_messages(
                previous_analyses=json.dumps(previous_analyses, indent=2),
                context="\n\n".join(context)
            )
            response = llm.invoke(messages)

            new_state = state.copy()
            new_state["messages"].append(response)
            new_state["results"]["Connections"] = response.content

            return new_state

        except Exception as e:
            return handle_error(state, f"Connections Analysis Error: {str(e)}")

    return analyze_connections

def create_workflow():
    """Create and configure the workflow"""
    workflow = StateGraph(VetState)

    # Add nodes
    workflow.add_node("process_data", create_data_processing_agent())
    workflow.add_node("analyze_history", create_history_analysis_agent())
    workflow.add_node("analyze_clinical", create_clinical_analysis_agent())
    workflow.add_node("diagnose", create_diagnostic_agent())
    workflow.add_node("specialist_summary", create_specialist_summary_agent())
    workflow.add_node("analyze_connections", create_connections_agent())

    # Set entry point
    workflow.set_entry_point("process_data")

    # Add edges
    workflow.add_edge("process_data", "analyze_history")
    workflow.add_edge("analyze_history", "analyze_clinical")
    workflow.add_edge("analyze_clinical", "diagnose")
    workflow.add_edge("diagnose", "specialist_summary")
    workflow.add_edge("specialist_summary", "analyze_connections")
    workflow.add_edge("analyze_connections", END)

    return workflow.compile()

def analyze_veterinary_case(file_path: str):
    """Main function to analyze a veterinary case"""
    try:
        print("Processing document...")
        document_chunks = process_pdf(file_path)

        initial_state = {
            "messages": [],
            "current_input": "\n\n".join(document_chunks),
            "medical_records": {},
            "results": {
                "DataProcessor": "",
                "HistoryAnalyzer": "",
                "ClinicalAnalyzer": "",
                "Diagnostics": "",
                "SpecialistSummary": "",
                "Connections": ""
            },
            "metadata": {},
            "context": []
        }

        print("Creating workflow...")
        app = create_workflow()

        print("Running analysis...")
        result = app.invoke(initial_state)

        return result

    except Exception as e:
        print(f"Error in analysis: {str(e)}")
        raise

# Main execution
if __name__ == "__main__":
    file_path = "./input.pdf"
    result = analyze_veterinary_case(file_path)

    # Print results
    print("\nAnalysis Results:")
    print("-" * 50)

    sections = [
        ("Data Processing", "DataProcessor"),
        ("History Analysis", "HistoryAnalyzer"),
        ("Clinical Analysis", "ClinicalAnalyzer"),
        ("Diagnostic Assessment", "Diagnostics"),
        ("Specialist Summary", "SpecialistSummary"),
        ("Connections Analysis", "Connections")
    ]

    for title, key in sections:
        print(f"\n{title}:")
        print("-" * 30)
        print(result["results"].get(key, "Not available"))

    if "errors" in result["results"]:
        print("\nErrors encountered:")
        for error in result["results"]["errors"]:
            print(f"- {error['message']} (at {error['timestamp']}")

Processing document...
Creating workflow...
Running analysis...

Analysis Results:
--------------------------------------------------

Data Processing:
------------------------------
### Patient Information:
- **Patient Name:** Jax Miracle
- **Patient ID:** CA2441B8
- **Species:** Canine Pomeranian
- **Breed:** Pomeranian
- **Sex:** Male Castrated
- **Age:** 4 years old
- **Date of Birth:** December 20, 2020
- **Weight:** 6.1 kg
- **Microchip #:** None

### Owner Information:
- **Owner Name:** Anna Miracle
- **Owner ID:** 24404F
- **Address:** 2497 Hillsboro Rd, Campbellsburg, KY 40011, USA
- **Phone:** (502) 532-6641
- **Email:** miracletricia@gmail.com

### Visit Information:
- **Reason for Visit:** Coughing
- **Veterinarian:** Dr. Grace Clark / Dr. Scott Rizzo
- **Check-In:** February 4, 2025, 9:00 AM
- **Check-Out:** February 4, 2025, 1:00 PM
- **Referring Practice:** Countryside Animal Hospital, La Grange Veterinarian, (502) 222-4663

### Medical History:
- **Symptoms:** Coughing,

In [30]:
# Install required packages
!pip install -q langchain langgraph langchain_openai openai python-dotenv pinecone langchain_community pypdf

import os
from typing import List, Dict, Tuple, Annotated, TypedDict, Union, Any, Optional, Literal
from datetime import datetime
from enum import Enum
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, AIMessage
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from pinecone import Pinecone
from google.colab import userdata
import json

# Configuration
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
PINECONE_API_KEY = userdata.get('PINECONE_API_KEY')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY

# Initialize core components
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0
)

pc = Pinecone(
    api_key=os.getenv("PINECONE_API_KEY")
)

index_name = "veterinary-ai-index"
embeddings = OpenAIEmbeddings()

# Core classes
class VetState(TypedDict):
    messages: List[Union[HumanMessage, AIMessage]]
    current_input: str
    medical_records: Dict[str, Any]
    results: Dict[str, Any]
    metadata: Dict[str, Any]
    context: List[str]

# Utility functions
def query_pinecone(query: str, top_k: int = 5) -> List[dict]:
    """Query Pinecone index for relevant context"""
    query_embedding = embeddings.embed_query(query)
    index = pc.Index(index_name)
    results = index.query(
        vector=query_embedding,
        top_k=top_k,
        include_metadata=True
    )
    return results.matches

def handle_error(state: VetState, error_message: str) -> VetState:
    new_state = state.copy()
    if "errors" not in new_state["results"]:
        new_state["results"]["errors"] = []
    new_state["results"]["errors"].append({
        "timestamp": datetime.now().isoformat(),
        "message": error_message,
        "context": state.get("context", [])
    })
    return new_state

def process_pdf(file_path: str) -> List[str]:
    """Process PDF document and split into chunks"""
    loader = PyPDFLoader(file_path)
    pages = loader.load_and_split()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
    )

    chunks = text_splitter.split_documents(pages)
    return [chunk.page_content for chunk in chunks]

def create_data_processing_agent():
    def process_data(state: VetState) -> VetState:
        try:
            relevant_docs = query_pinecone(state["current_input"])
            context = [doc.metadata.get("text", "") for doc in relevant_docs]
            context_text = "\n\n".join(context)

            prompt = ChatPromptTemplate.from_messages([
                ("system", """You are a veterinary data processing agent conducting detailed initial examination.
                Consider medical history and current findings: {context}

                EXTRACT AND STRUCTURE:
                1. Detailed History
                   - Duration and progression of symptoms
                   - Previous treatments and responses
                   - Specific symptom frequencies and patterns

                2. Current Presentation
                   - Vital signs with normal ranges
                   - Physical exam findings with abnormalities highlighted
                   - Current medications with exact dosing

                3. Key Clinical Findings
                   - Specific abnormalities noted
                   - Response to treatments
                   - Changes in condition

                4. Laboratory Results
                   - All test results with reference ranges
                   - Highlight abnormal values
                   - Note trending changes

                5. Diagnostic Imaging
                   - All imaging findings
                   - Compare with previous if available

                OUTPUT FORMAT:
                - Patient Information (detailed demographics)
                - Comprehensive History (including timeline)
                - Current Clinical Status
                - Specific Abnormal Findings
                - Treatment History with Responses
                - Detailed Lab Results Analysis"""),
                ("human", "Process this veterinary case data in detail: {input}")
            ])

            messages = prompt.format_messages(
                input=state["current_input"],
                context=context_text
            )
            response = llm.invoke(messages)

            new_state = state.copy()
            new_state["messages"].append(response)
            new_state["results"]["DataProcessor"] = response.content
            new_state["context"] = context

            return new_state

        except Exception as e:
            return handle_error(state, f"Data Processing Error: {str(e)}")

    return process_data

def create_history_analysis_agent():
    def analyze_history(state: VetState) -> VetState:
        try:
            processed_data = state["results"]["DataProcessor"]
            context = state.get("context", [])

            prompt = ChatPromptTemplate.from_messages([
                ("system", """You are a veterinary history specialist analyzing disease progression.

                DETAILED ANALYSIS REQUIREMENTS:
                1. Timeline Construction
                   - Initial onset of symptoms
                   - Progression pattern
                   - Treatment trials and responses
                   - Key clinical events

                2. Treatment History Analysis
                   - Specific medications used (with doses)
                   - Duration of treatments
                   - Response patterns
                   - Treatment failures or successes
                   - Side effects noted

                3. Clinical Pattern Recognition
                   - Disease progression rate
                   - Symptom clusters
                   - Trigger factors
                   - Complicating factors
                   - Risk factors

                4. Owner Compliance & Home Care
                   - Treatment adherence
                   - Home monitoring
                   - Environmental factors
                   - Quality of life impacts

                FORMAT:
                Provide chronological analysis with:
                - Specific dates
                - Treatment durations
                - Response patterns
                - Clinical correlations"""),
                ("human", "Analyze this case history with detailed timeline: {processed_data}")
            ])

            messages = prompt.format_messages(
                processed_data=processed_data,
                context="\n\n".join(context)
            )
            response = llm.invoke(messages)

            new_state = state.copy()
            new_state["messages"].append(response)
            new_state["results"]["HistoryAnalyzer"] = response.content

            return new_state

        except Exception as e:
            return handle_error(state, f"History Analysis Error: {str(e)}")

    return analyze_history

def create_clinical_analysis_agent():
    def analyze_clinical(state: VetState) -> VetState:
        try:
            history_analysis = state["results"]["HistoryAnalyzer"]
            context = state.get("context", [])

            prompt = ChatPromptTemplate.from_messages([
                ("system", """You are a veterinary clinical analyst providing comprehensive medical insights.
                Consider this historical context: {context}

                CLINICAL ANALYSIS REQUIREMENTS:
                1. Physical Examination Findings
                   - Detailed review of all systems
                   - Vital signs trends
                   - Specific abnormalities with clinical significance
                   - Compare to previous exams

                2. Laboratory Data Analysis
                   - Review all results with reference ranges
                   - Trend analysis of serial results
                   - Clinical correlation of abnormalities
                   - Impact on treatment decisions

                3. Diagnostic Imaging Review
                   - Detailed findings analysis
                   - Progressive changes noted
                   - Clinical correlation
                   - Impact on treatment plan

                4. Treatment Response Assessment
                   - Effectiveness of current therapy
                   - Side effects observed
                   - Need for adjustments
                   - Compliance factors

                FORMAT:
                Provide detailed analysis with:
                - Specific clinical findings
                - Laboratory result interpretations
                - Treatment response evaluation
                - Recommendations for adjustments"""),
                ("human", "Provide detailed clinical analysis for: {history_analysis}")
            ])

            messages = prompt.format_messages(
                history_analysis=history_analysis,
                context="\n\n".join(context)
            )
            response = llm.invoke(messages)

            new_state = state.copy()
            new_state["messages"].append(response)
            new_state["results"]["ClinicalAnalyzer"] = response.content

            return new_state

        except Exception as e:
            return handle_error(state, f"Clinical Analysis Error: {str(e)}")

    return analyze_clinical

def create_diagnostic_agent():
    def diagnose(state: VetState) -> VetState:
        try:
            clinical_analysis = state["results"]["ClinicalAnalyzer"]
            context = state.get("context", [])

            diagnostic_context = query_pinecone(clinical_analysis)
            additional_context = [doc.metadata.get("text", "") for doc in diagnostic_context]
            combined_context = context + additional_context

            prompt = ChatPromptTemplate.from_messages([
                ("system", """You are a veterinary diagnostician providing expert assessment.
                Consider all findings and clinical context: {context}

                DIAGNOSTIC PROCESS:
                1. Primary Problem Assessment
                   - Define and rank clinical issues
                   - Document supporting evidence
                   - Note symptom patterns and severity
                   - Assess disease stage/progression

                2. Differential Analysis
                   - Comprehensive differential list
                   - Evidence supporting/refuting each
                   - Most likely diagnosis with rationale
                   - Rule-outs with justification

                3. Diagnostic Plan
                   - Specific tests needed with rationale
                   - Expected findings
                   - Monitoring parameters
                   - Follow-up timeline

                4. Treatment Recommendations
                   - Current treatment assessment
                   - Specific medication adjustments
                   - Dosing recommendations
                   - Monitoring plan

                FORMAT:
                Provide detailed assessment with:
                - Definitive diagnoses
                - Supporting evidence
                - Specific recommendations
                - Follow-up plan"""),
                ("human", "Provide comprehensive diagnostic assessment for: {clinical_analysis}")
            ])

            messages = prompt.format_messages(
                clinical_analysis=clinical_analysis,
                context="\n\n".join(combined_context)
            )
            response = llm.invoke(messages)

            new_state = state.copy()
            new_state["messages"].append(response)
            new_state["results"]["Diagnostics"] = response.content
            new_state["context"] = combined_context

            return new_state

        except Exception as e:
            return handle_error(state, f"Diagnostic Error: {str(e)}")

    return diagnose

def create_specialist_summary_agent():
    def generate_summaries(state: VetState) -> VetState:
        try:
            diagnostic_assessment = state["results"]["Diagnostics"]
            context = state.get("context", [])

            prompt = ChatPromptTemplate.from_messages([
                ("system", """You are a veterinary specialist providing comprehensive case assessment.
                Consider all diagnostic findings: {context}

                SPECIALIST ASSESSMENT REQUIREMENTS:
                1. Disease Process Analysis
                   - Detailed pathophysiology
                   - Disease stage and severity
                   - Complicating factors
                   - Prognosis assessment

                2. Treatment Plan Development
                   - Comprehensive treatment strategy
                   - Medication recommendations with dosing
                   - Therapeutic goals
                   - Expected outcomes

                3. Client Education
                   - Disease process explanation
                   - Treatment rationale
                   - Home care instructions
                   - Warning signs to monitor

                4. Follow-up Planning
                   - Monitoring parameters
                   - Recheck schedule
                   - Treatment milestones
                   - Quality of life assessment

                FORMAT:
                1. Specialist-Level Assessment
                2. Owner-Friendly Summary
                3. Detailed Treatment Plan
                4. Monitoring Strategy
                5. Follow-up Schedule"""),
                ("human", "Generate comprehensive specialist assessment for: {diagnostic_assessment}")
            ])

            messages = prompt.format_messages(
                diagnostic_assessment=diagnostic_assessment,
                context="\n\n".join(context)
            )
            response = llm.invoke(messages)

            new_state = state.copy()
            new_state["messages"].append(response)
            new_state["results"]["SpecialistSummary"] = response.content

            return new_state

        except Exception as e:
            return handle_error(state, f"Specialist Summary Error: {str(e)}")

    return generate_summaries

def create_connections_agent():
    def analyze_connections(state: VetState) -> VetState:
        try:
            previous_analyses = {
                "data_processing": state["results"]["DataProcessor"],
                "history": state["results"]["HistoryAnalyzer"],
                "clinical": state["results"]["ClinicalAnalyzer"],
                "diagnostics": state["results"]["Diagnostics"],
                "specialist": state["results"]["SpecialistSummary"]
            }

            context = state.get("context", [])

            prompt = ChatPromptTemplate.from_messages([
                ("system", """You are a veterinary case integration specialist.
                Consider all case analyses: {context}

                INTEGRATION REQUIREMENTS:
                1. Disease Pattern Analysis
                   - Connect symptoms to pathophysiology
                   - Identify disease interactions
                   - Note complicating factors
                   - Assess disease synergy

                2. Treatment Integration
                   - Analyze treatment interactions
                   - Identify synergistic approaches
                   - Note potential conflicts
                   - Optimize therapeutic strategy

                3. Monitoring Integration
                   - Key parameters to track
                   - Critical thresholds
                   - Warning signs
                   - Quality of life indicators

                4. Long-term Management
                   - Disease progression expectations
                   - Preventive strategies
                   - Quality of life considerations
                   - Client support needs

                FORMAT:
                Provide integrated assessment with:
                1. Unified Case Theory
                2. Comprehensive Treatment Strategy
                3. Integrated Monitoring Plan
                4. Long-term Management Guidelines"""),
                ("human", "Provide comprehensive case integration analysis for: {previous_analyses}")
            ])

            messages = prompt.format_messages(
                previous_analyses=json.dumps(previous_analyses, indent=2),
                context="\n\n".join(context)
            )
            response = llm.invoke(messages)

            new_state = state.copy()
            new_state["messages"].append(response)
            new_state["results"]["Connections"] = response.content

            return new_state

        except Exception as e:
            return handle_error(state, f"Connections Analysis Error: {str(e)}")

    return analyze_connections

def create_workflow():
    """Create and configure the workflow"""
    workflow = StateGraph(VetState)

    # Add nodes
    workflow.add_node("process_data", create_data_processing_agent())
    workflow.add_node("analyze_history", create_history_analysis_agent())
    workflow.add_node("analyze_clinical", create_clinical_analysis_agent())
    workflow.add_node("diagnose", create_diagnostic_agent())
    workflow.add_node("specialist_summary", create_specialist_summary_agent())
    workflow.add_node("analyze_connections", create_connections_agent())

    # Set entry point
    workflow.set_entry_point("process_data")

    # Add edges
    workflow.add_edge("process_data", "analyze_history")
    workflow.add_edge("analyze_history", "analyze_clinical")
    workflow.add_edge("analyze_clinical", "diagnose")
    workflow.add_edge("diagnose", "specialist_summary")
    workflow.add_edge("specialist_summary", "analyze_connections")
    workflow.add_edge("analyze_connections", END)

    return workflow.compile()

def analyze_veterinary_case(file_path: str):
    """Main function to analyze a veterinary case"""
    try:
        print("Processing document...")
        document_chunks = process_pdf(file_path)

        initial_state = {
            "messages": [],
            "current_input": "\n\n".join(document_chunks),
            "medical_records": {},
            "results": {
                "DataProcessor": "",
                "HistoryAnalyzer": "",
                "ClinicalAnalyzer": "",
                "Diagnostics": "",
                "SpecialistSummary": "",
                "Connections": ""
            },
            "metadata": {},
            "context": []
        }

        print("Creating workflow...")
        app = create_workflow()

        print("Running analysis...")
        result = app.invoke(initial_state)

        return result

    except Exception as e:
        print(f"Error in analysis: {str(e)}")
        raise

# Main execution
if __name__ == "__main__":
    file_path = "./input.pdf"
    result = analyze_veterinary_case(file_path)

    # Print results
    print("\nAnalysis Results:")
    print("-" * 50)

    sections = [
        ("Data Processing", "DataProcessor"),
        ("History Analysis", "HistoryAnalyzer"),
        ("Clinical Analysis", "ClinicalAnalyzer"),
        ("Diagnostic Assessment", "Diagnostics"),
        ("Specialist Summary", "SpecialistSummary"),
        ("Connections Analysis", "Connections")
    ]

    for title, key in sections:
        print(f"\n{title}:")
        print("-" * 30)
        print(result["results"].get(key, "Not available"))

    if "errors" in result["results"]:
        print("\nErrors encountered:")
        for error in result["results"]["errors"]:
            print(f"- {error['message']} (at {error['timestamp']}")

Processing document...
Creating workflow...
Running analysis...

Analysis Results:
--------------------------------------------------

Data Processing:
------------------------------
**Patient Information:**
- Name: Jax Miracle
- ID: CA2441B8
- Species: Canine Pomeranian
- Gender: Male, Neutered
- Age: 4 years old
- Date of Birth: December 20, 2020
- Weight: 6.1 kg
- Microchip: None

**Owner Information:**
- Name: Anna Miracle
- ID: 24404F
- Address: 2497 Hillsboro Rd, Campbellsburg, KY 40011, USA
- Phone: (502) 532-6641
- Email: miracletricia@gmail.com

**Visit Information:**
- Reason for Visit: Coughing
- Veterinarian: Dr. Grace Clark / Dr. Scott Rizzo
- Check-In: February 4, 2025, 9:00 AM
- Check-Out: February 4, 2025, 1:00 PM
- Referring Practice: Countryside Animal Hospital

**Comprehensive History:**
- Jax has been experiencing coughing, hacking, and loud panting for an unspecified duration.
- No vomiting or diarrhea reported.
- Dry cough and normal feces observed.
- No changes i